In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import pytz

from post_processing.dataclass.data_aplose import DataAplose
from post_processing.utils.fpod_utils import cpod2aplose, usable_data_phase, dpm_to_dph,build_hour_range, fb_folder,csv_folder, meta_cut_aplose
from post_processing.utils.core_utils import json2df

### Pre-processing



In [ ]:
data = csv_folder(r"U:\A")
print(data.head())

df_1 = data[data["DPM"]==1]

df_aplose = cpod2aplose(df_1, pytz.utc, "CA4", "Commerson", extra_columns=["DPM", "deploy.name"])
print(df_aplose.head())

In [ ]:
json = Path(r"C:\Users\fouinel\Downloads\deployment_kerguelen.json")
metadatax = json2df(json_path=json)

metadatax["deploy.name"] = (metadatax["site.name"].astype(str) + "_" +
                           metadatax["campaign.name"].astype(str))

data = meta_cut_aplose(df_aplose, metadatax)

df_aplose.to_csv(r"U:\APLOSE_A1112.csv", index=False)

## Load data

In [ ]:
yaml_file = Path(r"resource\CPOD-FPOD_yaml.yml")
data_list = DataAplose.from_yaml(file=yaml_file)
print(data_list.df.head())

## Create a detection per hour dataframe



In [ ]:
dph = dpm_to_dph(data_list.df, extra_columns=["deploy.name"])

## Add the feeding buzzes

In [ ]:
fb = fb_folder(r"U:\fb_A_NBHF", "Commerson")
fb["Date heure"] = fb["start_datetime"].dt.floor("h")
fb = fb.groupby("Date heure")["Foraging"].sum().reset_index()

d_hour = dph.merge(fb[["Date heure","Foraging"]], on="Date heure", how="left")

In [ ]:
new = build_hour_range(dph)
final_df = new.merge(d_hour, on=["deploy.name", "Date heure"], how = "left")

final_df[["DPM","Foraging"]] = final_df[["DPM","Foraging"]].fillna(0)
print(final_df.head())

In [ ]:
final_df["Year"] = final_df["Date heure"].dt.year
final_df["Month"] = final_df["Date heure"].dt.month
final_df["Day"] = final_df["Date heure"].dt.day
final_df["Hour"] = final_df["Date heure"].dt.hour

final_df["FBR"] = final_df["Foraging"] / final_df["DPM"]

In [ ]:
final_df["FBR"] = final_df["FBR"].fillna(0)
final_df.to_csv(r"U:\Hours_DPM_FBUZZ_SiteA1112.csv", index=False)